In [1]:
import torch
import dgl
import numpy as np
import torch.nn as nn

from model_Boltzmann import MesoToMacroDecoder, CollisionOperatorBGK, BoltzmannUpdater

def get_macro(f_distribution, xi_velocities):
    density = f_distribution.mean(dim=1, keepdim=True)
    momentum = (f_distribution * xi_velocities).mean(dim=1, keepdim=True)
    velocity = momentum / (density + 1e-8)  # Avoid division by
    return density, momentum, velocity

Q_mesoscale = 5
min_velocity = 0.0
max_velocity = 10.0
dt = 1.0
num_nodes = 4
road_length = 5
device = torch.device("cpu")

u = torch.tensor([0, 1, 2, 3])
v = torch.tensor([1, 2, 3, 0])
graph = dgl.graph((u, v), num_nodes=num_nodes).to(device)
graph.edata['weight'] = torch.ones(graph.num_edges(), device=device).float() / road_length

f_initial = torch.zeros(num_nodes, Q_mesoscale, device=device)
f_initial[0, 1] = 10.0
f_initial[2, 4] = 10.0

xi_velocities = torch.linspace(min_velocity, max_velocity, Q_mesoscale, device=device)

decoder = MesoToMacroDecoder(Q_mesoscale, xi_velocities=xi_velocities)

collision_op = CollisionOperatorBGK(
    Q_mesoscale=Q_mesoscale,
    min_macrovelocity=min_velocity,
    max_macrovelocity=max_velocity,
    hidden_dim=64, # Default value from train_Boltzmann.py
    num_layers=5   # Default value from train_Boltzmann.py
).to(device)

updater = BoltzmannUpdater(
    Q_mesoscale=Q_mesoscale,
    dt=dt,
    min_macrovelocity=min_velocity,
    max_macrovelocity=max_velocity,
    base_graph=graph
)

node_position_embedding = torch.linspace(0, 1, num_nodes, device=device).unsqueeze(1)

source_term = torch.zeros_like(f_initial)

print("--- Initial State ---")
print(f"Initial f distribution:\n{f_initial.cpu().numpy()}")

initial_macro_velocity = decoder(f_initial)
print(f"Initial Macroscopic Velocity:\n{initial_macro_velocity.cpu().numpy()}")

initial_macro_features = torch.cat([initial_macro_velocity, torch.zeros_like(initial_macro_velocity)], dim=-1) # Assuming second feature is 0
collision_term, f_eq = collision_op(f_initial, initial_macro_features, node_position_embedding)
density_eq, momentum_eq, macro_velocity_eq = get_macro(f_eq, xi_velocities.unsqueeze(0))
print(f'Equilibrium density:\n{density_eq.cpu().numpy()}')
# print(f'Equilibrium momentum:\n{momentum_eq.cpu().numpy()}')
print(f'Equilibrium velocity:\n{macro_velocity_eq.cpu().numpy()}')

density, momentum, macro_velocity = get_macro(f_initial, xi_velocities.unsqueeze(0))
print(f'Initial density:\n{density.cpu().numpy()}')
# print(f'Initial momentum:\n{momentum.cpu().numpy()}')
print(f'Initial velocity:\n{macro_velocity.cpu().numpy()}')



# transport_term = updater.compute_transport_term(f_initial)

# f_next = updater(f_initial, collision_term, source_term)

# final_macro_velocity = decoder(f_next)

# print("\n--- Single Step Evolution ---")
# print(f"Transport Term:\n{transport_term.cpu().numpy()}")
# print(f"\nCollision Term (Omega):\n{collision_term.detach().cpu().numpy()}")

# print("\n--- Final State (after dt=1.0) ---")
# print(f"Next f distribution:\n{f_next.detach().cpu().numpy()}")
# print(f"\nFinal Macroscopic Velocity:\n{final_macro_velocity.detach().cpu().numpy()}")

/home/xlc_wsl2/miniconda3/envs/dgl_traffic/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Initial State ---
Initial f distribution:
[[ 0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0.  0.]]
Initial Macroscopic Velocity:
[[ 2.5]
 [ 0. ]
 [10. ]
 [ 0. ]]
Equilibrium density:
[[2.       ]
 [0.       ]
 [1.9999993]
 [0.       ]]
Equilibrium velocity:
[[2.4999995]
 [0.       ]
 [9.999972 ]
 [0.       ]]
Initial density:
[[2.]
 [0.]
 [2.]
 [0.]]
Initial velocity:
[[ 2.5]
 [ 0. ]
 [10. ]
 [ 0. ]]
